In [1]:
pip install selenium beautifulsoup4 pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 7.0 MB/s eta 0:00:00


In [2]:
import random
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# Настройки Chrome
options = Options()
options.add_argument("--headless")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36")

driver = webdriver.Chrome(options=options)

# --- Собираем ссылки на заведения со всех страниц ---
all_firm_links = set()
for page in range(1, 3):  # 2 страниц
    print(f"📄 Загружаем страницу {page}...")
    search_url = f"https://2gis.kz/almaty/search/%D0%93%D0%BE%D1%81%D1%82%D0%B8%D0%BD%D0%B8%D1%86%D1%8B/rubricId/269/page/{page}/?m=76.903857%2C43.246695%2F14.35"
    driver.get(search_url)
    time.sleep(5)

    # Прокрутка вниз
    for _ in range(3):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")
    place_links = soup.find_all("a", href=True)
    firm_links = [
        "https://2gis.kz" + link['href']
        for link in place_links
        if "/firm/" in link['href'] and "/tab" not in link['href']
    ]
    all_firm_links.update(firm_links)

# Выбор 20 случайных заведений
unique_links = list(all_firm_links)
random.shuffle(unique_links)
selected_links = unique_links[:25]
print(f"\n🔎 Найдено {len(unique_links)} заведений. Выбрано {len(selected_links)} случайных для парсинга.")

# --- Парсим отзывы по заведениям ---
reviews_data = []

for i, place_url in enumerate(selected_links, start=1):
    print(f"\n📍 Обрабатывается {i}/{len(selected_links)}: {place_url}")
    try:
        driver.get(place_url)
        time.sleep(5)

        # Переход на вкладку "Отзывы"
        try:
            review_tab = WebDriverWait(driver, 5).until(
                EC.element_to_be_clickable((By.XPATH, "//a[contains(@href, '/tab/reviews')]"))
            )
            review_tab.click()
            time.sleep(5)
        except:
            print("⚠ Нет вкладки 'Отзывы', пропускаем.")
            continue

        # Название заведения
        soup = BeautifulSoup(driver.page_source, "html.parser")
        name_tag = soup.find("h1", class_="_1x89xo5")
        name = name_tag.get_text(strip=True) if name_tag else "Не найдено"

        # Средняя оценка и количество оценок
        try:
            rating_value = driver.find_element(By.CLASS_NAME, "_y10azs").text.strip()
        except:
            rating_value = "Нет данных"
        try:
            rating_count = driver.find_element(By.CLASS_NAME, "_jspzdm").text.strip().split(" ")[0]
        except:
            rating_count = "Нет данных"

        # Прокрутка для подгрузки отзывов
        for _ in range(10):
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)

        # Отзывы (до 50)
        review_blocks = driver.find_elements(By.XPATH, "//div[@class='_1k5soqfl']")[:50]
        for block in review_blocks:
            try:
                user = block.find_element(By.XPATH, ".//span[@class='_16s5yj36']").get_attribute("title")
            except:
                user = "Не указано"
            try:
                date = block.find_element(By.XPATH, ".//div[@class='_a5f6uz']").text
            except:
                date = "Не указана"
            try:
                comment = block.find_element(By.XPATH, ".//div[@class='_49x36f']").text
            except:
                comment = "Комментарий отсутствует"
            try:
                stars = block.find_elements(By.XPATH, ".//div[@class='_1fkin5c']/span")
                rating = len(stars)
            except:
                rating = "Не оценено"

            reviews_data.append({
                "Пользователь": user,
                "Дата": date,
                "Комментарий": comment,
                "Рейтинг": rating,
                "Название заведения": name,
                "Средняя оценка": rating_value,
                "Кол-во оценок": rating_count
            })

    except Exception as e:
        print(f"❌ Ошибка при обработке {place_url}: {e}")
        continue

# --- Сохраняем результат ---
data = pd.DataFrame(reviews_data)
data.to_csv("topreviews.csv", index=False, encoding='utf-8-sig')
print(f"\n✅ Парсинг завершен! Сохранено {len(data)} отзывов.")
driver.quit()

📄 Загружаем страницу 1...
📄 Загружаем страницу 2...

🔎 Найдено 24 заведений. Выбрано 24 случайных для парсинга.

📍 Обрабатывается 1/24: https://2gis.kz/almaty/firm/70000001089553051?stat=eyJwYXJlbnRUYWJJZCI6ImQ5NGJkYjg3LWQ5MGItNDcwNC1iNzU1LTYzZjJlMmE5ODQ1NyIsImZvcmtFdmVudE9yZGluYWwiOjIsInVpRWxlbWVudCI6eyJuYW1lIjoicGxhY2VDYXJkTWluaSIsIm93bmVyTmFtZSI6InNlYXJjaFJlc3VsdHNMaXN0IiwicG9zaXRpb24iOjIyfSwicGxhY2VJdGVtIjp7ImVudGl0eSI6eyJpZCI6IjcwMDAwMDAxMDg5NTUzMDUxIiwidHlwZSI6ImJyYW5jaCIsInNlZ21lbnRJbmZvIjp7ImJhc2VMb2NhbGUiOiJydV9LWiIsInNlZ21lbnRJZCI6IjY3In19LCJnZW9Qb3NpdGlvbiI6eyJsb24iOjc2Ljk1NzYyNCwibGF0Ijo0My4yMTEwMTl9LCJoeWJyaWRFbnRpdGllcyI6W3siaWQiOiI5NDMwMDQ3Mzc1MTQ2Mjk5IiwidHlwZSI6ImJ1aWxkaW5nIn1dLCJhZHNTZWFyY2giOnRydWUsIm1haW5SdWJyaWMiOiIyNjkiLCJpc0RlbGV0ZWQiOmZhbHNlLCJvcmciOiI3MDAwMDAwMTA4OTU1MzA1MCIsImNvbnRleHRSdWJyaWMiOiIyNjkiLCJoYXNNb2RlbCI6ZmFsc2UsInJlc3VsdENsYXNzIjoyLCJwb3NpdGlvbiI6MjJ9LCJzZWFyY2hSZXN1bHRzTGlzdCI6eyJzaXplIjo1ODIsInNlYXJjaEhhc2giOiI0MTY0M2MwZC05N2I1LTQxODAtYTFlMC01Y

In [4]:
dd = pd.read_csv("topreviews.csv")
dd

,Пользователь,Дата,Комментарий,Рейтинг,Название заведения,Средняя оценка,Кол-во оценок
0,Не указано,"25 ноября 2024, отредактирован",Отличная локация - свежий воздух и 10 мин до ц...,5,Five Villas Hotel,4.6,108
1,Куаныш Кульбеков,7 апреля 2025,Отлично провели время в апартаментах . Приехал...,5,Five Villas Hotel,4.6,108
2,Svetlana Snatkova,"23 марта 2025, отредактирован","Хороший отель в тихом районе,есть семейные ном...",4,Five Villas Hotel,4.6,108
3,Gulzhamal Shaken,"7 марта 2025, отредактирован","Грязно, пыльно, не уютно. Но фотках одно, в жи...",2,Five Villas Hotel,4.6,108
4,Руслан Рахманов,6 марта 2025,Даурен жаксы екен.,5,Five Villas Hotel,4.6,108
...,...,...,...,...,...,...,...
1169,Жанна Исмаил,9 апреля 2025,Баян❤️и Фарида мы очень рады и счастливые 🥰 Вы...,5,Beis SPA Hotel & Resort,4.5,2565
1170,Жамиля Бултбаева,8 апреля 2025,Очень довольны🥰🙆🏻‍♀️😘\nАсема и Феруза спасибо ...,5,Beis SPA Hotel & Resort,4.5,2565
1171,Aizhan Kairatovna,8 апреля 2025,Прекрасное обслуживание! В спа девочки вообще ...,5,Beis SPA Hotel & Resort,4.5,2565
1172,Елдана Ертай,8 апреля 2025,Асема и Фируза спасибо вам девочки ! Так кайфа...,5,Beis SPA Hotel & Resort,4.5,2565
